# Imports

In [26]:
import datasets
import torch
import numpy as np
from torchvision.transforms.functional import pil_to_tensor

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load CIFAR-100

In [13]:
ds = datasets.load_dataset('cifar100',
                           cache_dir='./cache',
                           trust_remote_code=True)

print(ds)

Generating test split: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 238046.28 examples/s]

DatasetDict({
    train: Dataset({
        features: ['img', 'fine_label', 'coarse_label'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['img', 'fine_label', 'coarse_label'],
        num_rows: 10000
    })
})


# Get Image Size and Set Patch Shape

In [27]:
img = pil_to_tensor(ds['train'][0]['img'])

C, W, H = img.shape
Cp, Wp, Hp = C, 4, 4

img_shape = list(img.shape)
patch_shape = [Cp, Wp, Hp]
num_patches = int(np.prod(img_shape)/np.prod(patch_shape))

print(f'Image shape: {img.shape}')
print(f'Patch shape: {patch_shape}')
print(f'# Patches: {num_pataches}')

Image shape: torch.Size([3, 32, 32])
Patch shape: [3, 4, 4]
# Patches: 64


# Test How to Patchify Efficiently

In [48]:
true_patch = img[:Cp, :Wp, :Hp].flatten()
print(true_patch.shape)
#test_patch = torch.ones(true_patch.shape)
test_patch = img.unfold(1, Wp, Wp).unfold(2, Hp, Hp)
print(test_patch.shape)
test_patch = test_patch.permute(1, 2, 0, 3, 4).reshape((num_patches, -1))[0]
print(test_patch.shape)

print(true_patch == test_patch)

torch.Size([48])
torch.Size([3, 8, 8, 4, 4])
torch.Size([48])
tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True])


In [32]:
a = torch.randn(1, 3, 4, 6)
print(a.shape)
a.unfold(1, 1, 1).shape

torch.Size([1, 3, 4, 5])


torch.Size([1, 3, 4, 5, 1])